In [90]:
import sympy as sp
from sympy.utilities.lambdify import lambdify
import numpy as np

In [50]:

# Define symbolic variables for joint angles, velocities, accelerations
theta_1, theta_2, theta_3, theta_4, theta_5 = sp.symbols('theta_1 theta_2 theta_3 theta_4 theta_5')
dtheta_1, dtheta_2, dtheta_3, dtheta_4, dtheta_5 = sp.symbols('dtheta_1 dtheta_2 dtheta_3 dtheta_4 dtheta_5')
ddtheta_1, ddtheta_2, ddtheta_3, ddtheta_4, ddtheta_5 = sp.symbols('ddtheta_1 ddtheta_2 ddtheta_3 ddtheta_4 ddtheta_5')

In [51]:
# DH parameters
d_1, d_5 = sp.symbols('d_1 d_5')
a_2, a_3 = sp.symbols('a_2 a_3')
alpha = [90, 0, 0, 90, 0]

In [52]:
# Masses and inertia tensors of the links
m1, m2, m3, m4, m5 = sp.symbols('m1 m2 m3 m4 m5')
I1, I2, I3, I4, I5 = sp.symbols('I1 I2 I3 I4 I5')

In [53]:
# Gravity
g = sp.symbols('g')

In [54]:
# Helper function to create a transformation matrix from DH parameters
def dh_matrix(theta, d, a, alpha):
    alpha_rad = sp.rad(alpha)  # Convert alpha from degrees to radians
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha_rad), sp.sin(theta) * sp.sin(alpha_rad), a * sp.cos(theta)],
        [sp.sin(theta), sp.cos(theta) * sp.cos(alpha_rad), -sp.cos(theta) * sp.sin(alpha_rad), a * sp.sin(theta)],
        [0, sp.sin(alpha_rad), sp.cos(alpha_rad), d],
        [0, 0, 0, 1]
    ])

In [55]:

# Transformation matrices for each joint
A1 = dh_matrix(theta_1, d_1, 0, alpha[0])
A2 = dh_matrix(theta_2, 0, a_2, alpha[1])
A3 = dh_matrix(theta_3, 0, a_3, alpha[2])
A4 = dh_matrix(theta_4, 0, 0, alpha[3])
A5 = dh_matrix(theta_5, d_5, 0, alpha[4])

In [56]:
# Compute the overall transformation matrix by multiplying individual matrices
T1 = A1
T2 = T1 * A2
T3 = T2 * A3
T4 = T3 * A4
T5 = T4 * A5

In [57]:

# Position vectors of the center of mass of each link (assuming center of mass is at the middle of each link)
P1 = T1[:3, 3] / 2
P2 = T2[:3, 3] / 2
P3 = T3[:3, 3] / 2
P4 = T4[:3, 3] / 2
P5 = T5[:3, 3] / 2

In [58]:

# Velocity vectors of the center of mass of each link
V1 = P1.diff(theta_1) * dtheta_1 + P1.diff(theta_2) * dtheta_2 + P1.diff(theta_3) * dtheta_3 + P1.diff(theta_4) * dtheta_4 + P1.diff(theta_5) * dtheta_5
V2 = P2.diff(theta_1) * dtheta_1 + P2.diff(theta_2) * dtheta_2 + P2.diff(theta_3) * dtheta_3 + P2.diff(theta_4) * dtheta_4 + P2.diff(theta_5) * dtheta_5
V3 = P3.diff(theta_1) * dtheta_1 + P3.diff(theta_2) * dtheta_2 + P3.diff(theta_3) * dtheta_3 + P3.diff(theta_4) * dtheta_4 + P3.diff(theta_5) * dtheta_5
V4 = P4.diff(theta_1) * dtheta_1 + P4.diff(theta_2) * dtheta_2 + P4.diff(theta_3) * dtheta_3 + P4.diff(theta_4) * dtheta_4 + P4.diff(theta_5) * dtheta_5
V5 = P5.diff(theta_1) * dtheta_1 + P5.diff(theta_2) * dtheta_2 + P5.diff(theta_3) * dtheta_3 + P5.diff(theta_4) * dtheta_4 + P5.diff(theta_5) * dtheta_5


In [59]:
# Kinetic energy of each link
T1 = 0.5 * m1 * (V1.dot(V1)) + 0.5 * I1 * dtheta_1**2
T2 = 0.5 * m2 * (V2.dot(V2)) + 0.5 * I2 * dtheta_2**2
T3 = 0.5 * m3 * (V3.dot(V3)) + 0.5 * I3 * dtheta_3**2
T4 = 0.5 * m4 * (V4.dot(V4)) + 0.5 * I4 * dtheta_4**2
T5 = 0.5 * m5 * (V5.dot(V5)) + 0.5 * I5 * dtheta_5**2


In [60]:
# Total kinetic energy
T = T1 + T2 + T3 + T4 + T5

In [61]:
# Potential energy of each link
V1 = m1 * g * P1[2]
V2 = m2 * g * P2[2]
V3 = m3 * g * P3[2]
V4 = m4 * g * P4[2]
V5 = m5 * g * P5[2]

In [62]:

# Total potential energy
V = V1 + V2 + V3 + V4 + V5

In [63]:
# Lagrangian
L = T - V

In [64]:
# Generalized coordinates and their derivatives
q = sp.Matrix([theta_1, theta_2, theta_3, theta_4, theta_5])
dq = sp.Matrix([dtheta_1, dtheta_2, dtheta_3, dtheta_4, dtheta_5])
ddq = sp.Matrix([ddtheta_1, ddtheta_2, ddtheta_3, ddtheta_4, ddtheta_5])


In [65]:
# Euler-Lagrange equations
EOM = sp.Matrix([
    sp.diff(sp.diff(L, dq[i]), 't') - sp.diff(L, q[i]) for i in range(len(q))
])

In [66]:
# Substitute time derivatives of generalized coordinates
EOM = EOM.subs({
    q[i].diff('t'): dq[i] for i in range(len(q))
}).subs({
    dq[i].diff('t'): ddq[i] for i in range(len(dq))
})

In [67]:
# Simplify the equations of motion
EOM_simplified = sp.simplify(EOM)


In [68]:
# Print the simplified equations of motion
sp.init_printing(use_unicode=True)
print("Simplified Equations of Motion:")
sp.pprint(EOM_simplified)

Simplified Equations of Motion:
⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢a₃⋅g⋅m₃⋅cos(θ₂ + θ₃)   a₃⋅g⋅m₄⋅cos(θ₂ + θ₃)               ⎛          2       
⎢──────────────────── + ──────────────────── + 0.125⋅a₃⋅m₃⋅⎝a₂⋅dtheta₁ ⋅sin(θ₃
⎢         2                      2                                            
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                   

In [69]:
# Torque equations
torques = [EOM_simplified[i].subs(ddq[i], 0) for i in range(len(ddq))]

In [72]:
# Print the torque equations
print("\nTorque Equations:")
for i in range(len(torques)):
    print(f"Tau_{i+1} = ")
    sp.pprint(torques[i])
    print()


Torque Equations:
Tau_1 = 
0

Tau_2 = 
        2        2                a₂⋅g⋅m₂⋅cos(θ₂)                2    ⎛  2    
0.125⋅a₂ ⋅dtheta₁ ⋅m₂⋅sin(2⋅θ₂) + ─────────────── + 0.125⋅dtheta₁ ⋅m₃⋅⎝a₂ ⋅sin
                                         2                                    

                                    2                 ⎞                2    ⎛ 
(2⋅θ₂) + 2⋅a₂⋅a₃⋅sin(2⋅θ₂ + θ₃) + a₃ ⋅sin(2⋅θ₂ + 2⋅θ₃)⎠ + 0.125⋅dtheta₁ ⋅m₄⋅⎝a
                                                                              

 2                                        2                 ⎞               2 
₂ ⋅sin(2⋅θ₂) + 2⋅a₂⋅a₃⋅sin(2⋅θ₂ + θ₃) + a₃ ⋅sin(2⋅θ₂ + 2⋅θ₃)⎠ + 0.25⋅dtheta₁ ⋅
                                                                              

   ⎛  2                                                                       
m₅⋅⎝a₂ ⋅sin(θ₂)⋅cos(θ₂) + a₂⋅a₃⋅sin(θ₂)⋅cos(θ₂ + θ₃) + a₂⋅a₃⋅sin(θ₂ + θ₃)⋅cos(
                                                                              

        

In [81]:

# Define the constant velocities and accelerations
const_velocities = {
    dtheta_1: 0,
    dtheta_2: 0,
    dtheta_3: 0,
    dtheta_4: 0,
    dtheta_5: 0
}

const_accelerations = {
    ddtheta_1: 0,
    ddtheta_2: 0,
    ddtheta_3: 0,
    ddtheta_4: 0,
    ddtheta_5: 0
}

# Torque equations without considering accelerations and velocities (set to 0)
torques_no_vel_acc = [EOM_simplified[i].subs(const_accelerations).subs(const_velocities) for i in range(len(ddq))]

# Given DH Parameters and system constants
params = {
    d_1: 0.1,
    d_5: 0.1,
    a_2: 0.5,
    a_3: 0.5,
    m1: 1,
    m2: 1,
    m3: 1,
    m4: 1,
    m5: 1,
    I1: 0.1,
    I2: 0.1,
    I3: 0.1,
    I4: 0.1,
    I5: 0.1,
    g: 9.81,
    theta_1: 0,
    theta_2: 0,
    theta_3: 0,
    theta_4: 0,
    theta_5: 0
}

# Substitute the given parameters into the torque equations
torques_gravity = [torque.subs(params) for torque in torques_no_vel_acc]

# Sum of torques due to gravity
total_torque_gravity = sum(torques_gravity)

# Print the torque equations due to gravity
print("\nTorque Equations due to Gravity:")
for i in range(len(torques_gravity)):
    print(f"Tau_{i+1} = ")
    sp.pprint(torques_gravity[i])
    print()

# Print the total torque due to gravity
print("Total Torque due to Gravity:")
sp.pprint(total_torque_gravity)


Torque Equations due to Gravity:
Tau_1 = 
0

Tau_2 = 
17.1675000000000

Tau_3 = 
7.35750000000000

Tau_4 = 
0

Tau_5 = 
0

Total Torque due to Gravity:
24.5250000000000


In [87]:

# Define the ranges for each joint in degrees and convert to radians
theta_1_range = np.radians(np.linspace(0, 360, 2))
theta_2_range = np.radians(np.linspace(-90, 90, 2))
theta_3_range = np.radians(np.linspace(-90, 90, 2))
theta_4_range = np.radians(np.linspace(-90, 90, 2))
theta_5_range = np.radians(np.linspace(0, 360, 2))

# Initialize a list to store the maximum torques
max_torques = [0] * len(torques_gravity)

# Iterate over all combinations of joint angles
for theta1 in theta_1_range:
    for theta2 in theta_2_range:
        for theta3 in theta_3_range:
            for theta4 in theta_4_range:
                for theta5 in theta_5_range:
                    # Substitute the joint angles into the torque equations
                    current_torques = [torque.subs({theta_1: theta1, theta_2: theta2, theta_3: theta3, theta_4: theta4, theta_5: theta5}) for torque in torques_gravity]
                    # Update the maximum torques if current torques are greater
                    for i in range(len(max_torques)):
                        max_torques[i] = max(max_torques[i], abs(current_torques[i]))

# Print the maximum torques for each joint
print("Maximum Torques for Each Joint:")
for i in range(len(max_torques)):
    print(f"Max Tau_{i+1} = {max_torques[i]}")

Maximum Torques for Each Joint:
Max Tau_1 = 0
Max Tau_2 = 17.1675000000000
Max Tau_3 = 7.35750000000000
Max Tau_4 = 0
Max Tau_5 = 0


In [106]:

# Define the ranges for each joint in degrees and convert to radians
theta_1_range = np.radians(np.linspace(0, 360, 10))
theta_2_range = np.radians(np.linspace(-90, 90, 10))
theta_3_range = np.radians(np.linspace(-90, 90, 10))
theta_4_range = np.radians(np.linspace(-90, 90, 10))
theta_5_range = np.radians(np.linspace(0, 360, 10))

# Define the ranges for velocities and accelerations
dtheta_range = np.linspace(-1, 1, 3)  # Velocity range from -1 to 1 rad/s
ddtheta_range = np.linspace(-10, 10, 3)  # Acceleration range from -10 to 10 rad/s^2

# Convert symbolic expressions to numerical functions
torque_funcs = [lambdify(
    (theta_1, theta_2, theta_3, theta_4, theta_5, dtheta_1, dtheta_2, dtheta_3, dtheta_4, dtheta_5, ddtheta_1, ddtheta_2, ddtheta_3, ddtheta_4, ddtheta_5),
    torque
) for torque in torques_gravity]

# Initialize a list to store the maximum torques
max_torques = [0] * len(torque_funcs)

# Sampling method
num_samples = 10000  # Number of samples to take

# Generate random samples
np.random.seed(0)  # For reproducibility
samples = np.random.rand(num_samples, 15)

# Map samples to the respective ranges
samples[:, 0] = samples[:, 0] * (theta_1_range.max() - theta_1_range.min()) + theta_1_range.min()
samples[:, 1] = samples[:, 1] * (theta_2_range.max() - theta_2_range.min()) + theta_2_range.min()
samples[:, 2] = samples[:, 2] * (theta_3_range.max() - theta_3_range.min()) + theta_3_range.min()
samples[:, 3] = samples[:, 3] * (theta_4_range.max() - theta_4_range.min()) + theta_4_range.min()
samples[:, 4] = samples[:, 4] * (theta_5_range.max() - theta_5_range.min()) + theta_5_range.min()

samples[:, 5:10] = samples[:, 5:10] * (dtheta_range.max() - dtheta_range.min()) + dtheta_range.min()
samples[:, 10:15] = samples[:, 10:15] * (ddtheta_range.max() - ddtheta_range.min()) + ddtheta_range.min()

# Iterate over the sampled combinations
for combination in samples:
    current_torques = [func(*combination) for func in torque_funcs]
    for i in range(len(max_torques)):
        max_torques[i] = max(max_torques[i], abs(current_torques[i]))

# Print the maximum torques for each joint
print("Maximum Dynamic Torques for Each Joint:")
for i in range(len(max_torques)):
    print(f"Max Tau_{i+1} = {max_torques[i]}")

Maximum Dynamic Torques for Each Joint:
Max Tau_1 = 0
Max Tau_2 = 17.613564030873924
Max Tau_3 = 8.086750091715075
Max Tau_4 = 0.5637950042142448
Max Tau_5 = 0.9999415656117772
